# BEDBASE workflow tutorial

This demo demonstrates how to process, analyze, visualize, and serve BED files. The process has 3 steps: First, individual BED files are analyzed using the [bedstat](https://github.com/databio/bedstat) pipeline. Second, BED files are grouped and then analyzed as groups using the [bedbuncher](https://github.com/databio/bedbuncher) pipeline. Finally, the BED files, along with statistics, plots, and grouping information, is served via a web interface and RESTful API using the [bedhost](https://github.com/databio/bedhost/tree/master) package.

**Glossary of terms:**

- *bedfile*: a tab-delimited file with one genomic region per line. Each genomic region is decribed by 3 required columns: chrom, start and end.
- *bedset*: a collection of BED files grouped by with a shared biological, experimental, or logical criterion.


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Preparation" data-toc-modified-id="1.-Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>1. Preparation</a></span></li><li><span><a href="#2.-BEDSTAT:-Generate-statistics-and-plots-of-BED-files" data-toc-modified-id="2.-BEDSTAT:-Generate-statistics-and-plots-of-BED-files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2. BEDSTAT: Generate statistics and plots of BED files</a></span><ul class="toc-item"><li><span><a href="#Get-a-PEP-describing-the-bedfiles-to-process" data-toc-modified-id="Get-a-PEP-describing-the-bedfiles-to-process-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Get a PEP describing the bedfiles to process</a></span></li><li><span><a href="#Install-bedstat-dependencies" data-toc-modified-id="Install-bedstat-dependencies-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Install bedstat dependencies</a></span></li><li><span><a href="#Inititiate-a-local-elasticsearch-cluster" data-toc-modified-id="Inititiate-a-local-elasticsearch-cluster-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Inititiate a local elasticsearch cluster</a></span></li><li><span><a href="#Run-bedstat--on-the-demo-PEP" data-toc-modified-id="Run-bedstat--on-the-demo-PEP-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Run bedstat  on the demo PEP</a></span></li></ul></li><li><span><a href="#3.-BEDBUNCHER:-Create-bedsets-and-their-respective-statistics" data-toc-modified-id="3.-BEDBUNCHER:-Create-bedsets-and-their-respective-statistics-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>3. BEDBUNCHER: Create bedsets and their respective statistics</a></span><ul class="toc-item"><li><span><a href="#Create-a-new-PEP-describing-the-bedset-name-and-specific-JSON-query" data-toc-modified-id="Create-a-new-PEP-describing-the-bedset-name-and-specific-JSON-query-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create a new PEP describing the bedset name and specific JSON query</a></span></li><li><span><a href="#Create-outputs-directory-and-install-bedbuncher-CML-dependencies" data-toc-modified-id="Create-outputs-directory-and-install-bedbuncher-CML-dependencies-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Create outputs directory and install bedbuncher CML dependencies</a></span></li><li><span><a href="#Run-bedbuncher-using-Looper" data-toc-modified-id="Run-bedbuncher-using-Looper-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Run bedbuncher using Looper</a></span></li></ul></li><li><span><a href="#4.-BEDHOST:--Serve-BED-files-and-API-to-explore-pipeline-outputs" data-toc-modified-id="4.-BEDHOST:--Serve-BED-files-and-API-to-explore-pipeline-outputs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>4. BEDHOST:  Serve BED files and API to explore pipeline outputs</a></span></li></ul></div>

## 1. Preparation 

First, we will create a tutorial directory where we'll store the bedbase pipelines and files to be processed. We'll also need to create an environment variable that points to the tutorial directory (we'll need this variable later). 

In [1]:
mkdir bedbase_tutorial
cd bedbase_tutorial
export BBTUTORIAL=`pwd`

Download some example BED files:

In [2]:
wget http://big.databio.org/example_data/bedbase_tutorial/bed_files.tar.gz     

--2020-06-08 11:25:06--  http://big.databio.org/example_data/bedbase_tutorial/bed_files.tar.gz
Resolving big.databio.org (big.databio.org)... 128.143.245.182, 128.143.245.181
Connecting to big.databio.org (big.databio.org)|128.143.245.182|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44549692 (42M) [application/octet-stream]
Saving to: ‘bed_files.tar.gz’

bed_files.tar.gz    100%[===================>]  42.49M  17.9MB/s    in 2.4s    

2020-06-08 11:25:09 (17.9 MB/s) - ‘bed_files.tar.gz’ saved [44549692/44549692]



The downloaded files are compressed so we'll need to untar them:

In [3]:
tar -zxvf bed_files.tar.gz

bed_files/
bed_files/GSE105587_ENCFF018NNF_conservative_idr_thresholded_peaks_GRCh38.bed.gz
bed_files/GSM2423312_ENCFF155HVK_peaks_GRCh38.bed.gz
bed_files/GSE105977_ENCFF617QGK_optimal_idr_thresholded_peaks_GRCh38.bed.gz
bed_files/GSE91663_ENCFF316ASR_peaks_GRCh38.bed.gz
bed_files/GSM2423313_ENCFF722AOG_peaks_GRCh38.bed.gz
bed_files/GSM2827349_ENCFF196DNQ_peaks_GRCh38.bed.gz
bed_files/GSE91663_ENCFF553KIK_optimal_idr_thresholded_peaks_GRCh38.bed.gz
bed_files/GSE91663_ENCFF319TPR_conservative_idr_thresholded_peaks_GRCh38.bed.gz
bed_files/GSE105977_ENCFF937CGY_peaks_GRCh38.bed.gz
bed_files/GSM2827350_ENCFF928JXU_peaks_GRCh38.bed.gz
bed_files/GSE105977_ENCFF793SZW_conservative_idr_thresholded_peaks_GRCh38.bed.gz


Additionally, we'll download a matrix we need to provide if we wish to plot the tissue specificity of our set of genomic ranges:

In [4]:
wget http://big.databio.org/open_chromatin_matrix/openSignalMatrix_hg38_percentile99_01_quantNormalized_round4d.txt.gz

--2020-06-08 11:25:42--  http://big.databio.org/open_chromatin_matrix/openSignalMatrix_hg38_quantileNormalized_round4.txt.gz
Resolving big.databio.org (big.databio.org)... 128.143.245.181, 128.143.245.182
Connecting to big.databio.org (big.databio.org)|128.143.245.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235485550 (225M) [application/octet-stream]
Saving to: ‘openSignalMatrix_hg38_quantileNormalized_round4.txt.gz’

openSignalMatrix_hg 100%[===================>] 224.58M  3.80MB/s    in 54s     

2020-06-08 11:26:36 (4.13 MB/s) - ‘openSignalMatrix_hg38_quantileNormalized_round4.txt.gz’ saved [235485550/235485550]



Lastly, we'll download the 3 core pipelines and tools needed to complete this tutorial: `bedstat`, `bedbuncher` and `bedhost`

In [5]:
git clone git@github.com:databio/bedbase
git clone git@github.com:databio/bedstat
git clone git@github.com:databio/bedbuncher
git clone git@github.com:databio/bedhost

Cloning into 'bedbase'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 2), reused 27 (delta 2), pack-reused 0
Receiving objects: 100% (27/27), 12.75 KiB | 12.75 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Cloning into 'bedstat'...
remote: Enumerating objects: 490, done.
remote: Total 490 (delta 0), reused 0 (delta 0), pack-reused 490
Receiving objects: 100% (490/490), 70.39 KiB | 1.50 MiB/s, done.
Resolving deltas: 100% (231/231), done.
Cloning into 'bedbuncher'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 350 (delta 84), reused 104 (delta 43), pack-reused 196
Receiving objects: 100% (350/350), 71.34 KiB | 2.97 MiB/s, done.
Resolving deltas: 100% (192/192), done.
Cloning into 'bedhost'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (325

## 2. BEDSTAT: Generate statistics and plots of BED files 

### Get a PEP describing the bedfiles to process

The first step is to process the BED files using the `bedstat` pipeline, which computes statistics and makes plots for each individual BED file. To begin, we'll need some annotation information for our BED files to load. We'll use the standard [PEP](https://pepkit.github.io/) format for the annotation, which consists of 1) a sample table (.csv) that annotates the files, and 2) a project config.yaml file that points to the sample annotation sheet. The config file also has other components, such as derived attributes, that in this case point to the bedfiles to be processed. Here is the PEP config file for this example project. It includes annotation information for each BED file, and also points to the `.bed.gz` files using derived attributes `output_file_path` and `yaml_file`.

In [6]:
cat bedbase/tutorial_files/PEPs/bedstat_config.yaml

pep_version: 2.0.0
sample_table: bedstat_annotation_sheet.csv

looper:
    output-dir: $BBTUTORIAL/outputs/bedstat_output/bedstat_pipeline_logs 

sample_modifiers:
  append:
    bedbase_config: $BBTUTORIAL/bedbase/tutorial_files/bedbase_configuration.yaml
    pipeline_interfaces: $BBTUTORIAL/bedstat/pipeline_interface_new.yaml
    output_file_path: OUTPUT
    yaml_file: SAMPLE_YAML
    open_signal_matrix: MATRIX
  derive:
    attributes: [output_file_path, yaml_file, open_signal_matrix]
    sources:
      OUTPUT: "$BBTUTORIAL/bed_files/{file_name}" 
      SAMPLE_YAML: "$BBTUTORIAL/outputs/bedstat_pipeline_logs/submission/{sample_name}.yaml"
      MATRIX: "$BBTUTORIAL/openSignalMatrix_{genome}_quantileNormalized_round4.txt.gz"

### Install bedstat dependencies

`bedstat` is a [pypiper](http://code.databio.org/pypiper/) pipeline that generates statistics and plots of bedfiles. Additionally, `bedstat` uses [bbconf](https://github.com/databio/bbconf), the bedbase configuration manager which implements convenience methods for interacting with an Elasticsearch database, where our file metadata will be placed. These and the appropriate R dependencies can be installed as follows:

In [8]:
pip install -r bedstat/requirements.txt --user > requirements_log.txt

: 1

Install R dependencies

In [32]:
Rscript bedstat/scripts/installRdeps.R > R_deps.txt

There's an additional dependency needed by `bedstat` if we wish to calculate and plot the GC content of our bedfiles. Depending on the genome assemblies of the files listed on a PEP, the appropriate BSgenome packages should be installed. The following is an example of how we can do so:

In [7]:
cat bedbase/tutorial_files/scripts/BSgenome_install.R

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("BSgenome.Hsapiens.UCSC.hg38.masked")

In [31]:
Rscript bedbase/tutorial_files/scripts/BSgenome_install.R > BSgenome.txt

We'll need to create a directory where we can store the stats and plots generated by `bedstat`. Additionally, we'll create a directory where we can store log and metadata files that we'll need later on.

In [8]:
mkdir -p outputs/bedstat_output/bedstat_pipeline_logs

In order to use `bbconf`, we'll need to create a minimal configuration.yaml file. The path to this configuration file can be stored in the environment variable `$BEDBASE`.

In [9]:
cat bedbase/tutorial_files/bedbase_configuration.yaml

path:
  bedstat_output: $BBTUTORIAL/outputs/bedstat_output
  bedbuncher_output: $BBTUTORIAL/outputs/bedbuncher_output

database:
  host: localhost
  bed_index: bed_index
  bedset_index: bedset_index

server:
  host: 0.0.0.0
  port: 8000


### Inititiate a local PostgreSQL instance

In addition to generate statistics and plots, `bedstat` inserts JSON formatted metadata into relational [PostgreSQL] database. 

If you don't have docker installed, you can install it with `sudo apt-get update && apt-get install docker-engine -y`.

Now, create a persistent volume to house PostgreSQL data:

In [9]:
docker volume create postgres-data

es-data


Spin up a `postgres` container. Provide required environment variables (need to match the settings in bedbase configuration file) and bind the created docker volume to `/var/lib/postgresql/data` path in the container:

In [13]:
docker run -d --name bedbase-postgres -p 5432:5432 -e POSTGRES_PASSWORD=bedbasepassword -e POSTGRES_USER=postgres -e POSTGRES_DB=postgres -v postgres-data:/var/lib/postgresql/data postgres

e1107272f4eda0c5cff262d71881600e76cff4a83c3438f6a786e89e17c6b72f


### Run bedstat  on the demo PEP
To run `bedstat` and the other required pipelines in this tutorial, we will rely on the pipeline submission engine [looper](http://looper.databio.org/en/latest/), which can be installed as follows:

In [ ]:
pip install looper --user

In order to establish a modular connection between a project and a pipeline, we'll need to create a [pipeline interface](http://looper.databio.org/en/latest/linking-a-pipeline/) file, which tells looper how to run the pipeline. 

In [10]:
cat bedstat/pipeline_interface_new.yaml

pipeline_name: BEDSTAT
pipeline_type: sample
path: pipeline/bedstat.py
input_schema: http://schema.databio.org/pipelines/bedstat.yaml
command_template: >
  {pipeline.path}
  --bedfile {sample.output_file_path}
  --genome {sample.genome}
  --sample-yaml {sample.yaml_file}
  {% if sample.bedbase_config is defined %} --bedbase-config {sample.bedbase_config} {% endif %}
  {% if sample.open_signal_matrix is defined %} --open-signal-matrix {sample.open_signal_matrix} {% endif %}


Once we have properly linked our project to the pipeline of interest, in this case` bedstat`, we simply need to point the `looper run` command our `PEP` config file. Additionally, if the bedbase configuration file location is not stored in the `$BEDBASE` variable, we can pass it to `looper` as an additional argument:

In [18]:
looper run bedbase/tutorial_files/PEPs/bedstat_config.yaml --package local \
--command-extra="-R" > outputs/bedstat_output/bedstat_pipeline_logs/looper_logs.txt

Looper version: 1.2.0
Command: run
Activating compute package 'local'
## [1 of 11] sample: bedbase_demo_db1; pipeline: BEDSTAT
Writing script to /home/jev4xy/Desktop/bedbase_tutorial/outputs/bedstat_output/bedstat_pipeline_logs/submission/BEDSTAT_bedbase_demo_db1.sub
Job script (n=1; 0.00Gb): /home/jev4xy/Desktop/bedbase_tutorial/outputs/bedstat_output/bedstat_pipeline_logs/submission/BEDSTAT_bedbase_demo_db1.sub
Established connection with Elasticsearch: localhost
## [2 of 11] sample: bedbase_demo_db2; pipeline: BEDSTAT
Writing script to /home/jev4xy/Desktop/bedbase_tutorial/outputs/bedstat_output/bedstat_pipeline_logs/submission/BEDSTAT_bedbase_demo_db2.sub
Job script (n=1; 0.00Gb): /home/jev4xy/Desktop/bedbase_tutorial/outputs/bedstat_output/bedstat_pipeline_logs/submission/BEDSTAT_bedbase_demo_db2.sub
Established connection with Elasticsearch: localhost
## [3 of 11] sample: bedbase_demo_db3; pipeline: BEDSTAT
Writing script to /home/jev4xy/Desktop/bedbase_tutorial/outputs/bedstat_o

Just for informative purposes, we can inspect how `bedstat` operates on each bedfile:

In [12]:
head outputs/bedstat_output/bedstat_pipeline_logs/looper_logs.txt

Compute node: cphg-51ksmr2
Start time: 2020-06-08 11:28:33
### Pipeline run code and environment:

*              Command:  `/home/jev4xy/Desktop/bedbase_tutorial/bedstat/pipeline/bedstat.py --bedfile /home/jev4xy/Desktop/bedbase_tutorial/bed_files/GSE105587_ENCFF018NNF_conservative_idr_thresholded_peaks_GRCh38.bed.gz --genome hg38 --sample-yaml /home/jev4xy/Desktop/bedbase_tutorial/outputs/bedstat_output/bedstat_pipeline_logs/submission/bedbase_demo_db1.yaml --bedbase-config /home/jev4xy/Desktop/bedbase_tutorial/bedbase/tutorial_files/bedbase_configuration.yaml --open-signal-matrix /home/jev4xy/Desktop/bedbase_tutorial/openSignalMatrix_hg38_quantileNormalized_round4.txt.gz -R`
*         Compute host:  cphg-51ksmr2
*          Working dir:  /home/jev4xy/Desktop/bedbase_tutorial
*            Outfolder:  /home/jev4xy/Desktop/bedbase_tutorial/outputs/bedstat_output/78c0e4753d04b238fc07e4ebe5a02984/
*  Pipeline started at:   (06-08 11:28:33) elapsed: 0.0 _TIME_



After the previous steps have been executed, our bedfiles should be available for query on our local Elasticsearch cluster. Files can be queried using the `bedbuncher` pipeline described in the below section. 


## 3. BEDBUNCHER: Create bedsets and their respective statistics 

### Create a new PEP describing the bedset name and specific JSON query 

Now that we've processed several individual BED files, we'll turn to the next task: grouping them together into collections of BED files, which we call *bedsets*. For this, we use the `bedbuncher` pipeline, which produces outputs for each bedset, such as a bedset PEP, bedset-level statistics and plots, and an `IGD` database. To run `bedbuncher`, we will need another PEP describing each bedset. Though the annotation sheet below specifies attributes for one bedset, you can create as many as you wish using additional rows. For each bedset, you need to provide the name of a JSON file containing the query to retrieve certain BED files. The JSONquery_name string will be used to build a sample derived attribute that points to the location of the JSON file. 

The following example PEP shows the attributes we need to provide for each bedset and the config.yaml file that will grab the files needed to run `bedbuncher`:

In [13]:
cat bedbase/tutorial_files/PEPs/bedbuncher_query.csv

sample_name,bedset_name,JSONquery_name,bbconfig_name,JSONquery_path,bedbase_config
bedset1,bedbase_demo_bedset,test_query,bedbase_configuration,source1,source2


In [14]:
cat bedbase/tutorial_files/PEPs/bedbuncher_config.yaml

pep_version: 2.0.0
sample_table: bedbuncher_query.csv

looper:
    output_dir: $BBTUTORIAL/outputs/bedbuncher_output/bedbuncher_pipeline_logs

sample_modifiers:
  append:
    pipeline_interfaces: $BBTUTORIAL/bedbuncher/pipeline_interface.yaml 
  derive:
    attributes: [JSONquery_path, bedbase_config]
    sources:
      source1: $BBTUTORIAL/bedbuncher/tests/{JSONquery_name}.json
      source2: $BBTUTORIAL/bedbase/tutorial_files/{bbconfig_name}.yaml


The Elasticsearch query format is complex and is difficult to create by hand. However, if you have a bedhost instance running, the JSON file contents can be generated using the bedhost graphical search interface. Just specify search criteria of interest and click "Get Elasticsearch query" button. This will generate a query string that looks something like this:

In [15]:
cat bedbuncher/tests/test_query.json

{
  "bool": {
    "must": [
      {
        "range": {
          "gc_content": {
            "gt": 0.1
          }
        }
      }
    ]
  }
}


###  Create outputs directory and install bedbuncher CML dependencies

We need a folder where we can store bedset related outputs. Though not required, we'll also create a directory where we can store the `bedbuncher` pipeline logs. 

In [16]:
mkdir -p outputs/bedbuncher_output/bedbuncher_pipeline_logs

One of the feats of `bedbuncher` includes [IGD](https://github.com/databio/IGD) database creation from the files in the bedset. `IGD` can be installed by cloning the repository from github, executing the make file to create the binary, and pointing the binary location with the `$PATH` environment variable. 

In [44]:
git clone git@github.com:databio/IGD
cd IGD
make > igd_make_log.txt 2>&1
cd ..

export PATH=$BBTUTORIAL/IGD/bin/:$PATH

Cloning into 'IGD'...


### Run bedbuncher using Looper 

Once we have cloned the `bedbuncher` repository, set our local Elasticsearch cluster and created the `iGD` binary, we can run the pipeline by pointing `looper run` to the appropriate `PEP` config file. As mentioned earlier, if the path to the bedbase configuration file has been stored in the `$BEDBASE` environment variable, it's not neccesary to pass the `--bedbase-config` argument. 

In [27]:
looper run  bedbase/tutorial_files/PEPs/bedbuncher_config.yaml  --package local \
--command-extra="-R" > outputs/bedbuncher_output/bedbuncher_pipeline_logs/looper_logs.txt

Looper version: 1.2.0
Command: run
Activating compute package 'local'
## [1 of 1] sample: bedset1; pipeline: BEDBUNCHER
Writing script to /home/jev4xy/Desktop/bedbase_tutorial/outputs/bedbuncher_output/bedbuncher_pipeline_logs/submission/BEDBUNCHER_bedset1.sub
Job script (n=1; 0.00Gb): /home/jev4xy/Desktop/bedbase_tutorial/outputs/bedbuncher_output/bedbuncher_pipeline_logs/submission/BEDBUNCHER_bedset1.sub

Looper finished
Samples valid for job generation: 1 of 1
Commands submitted: 1 of 1
Jobs submitted: 1


## 4. BEDHOST:  Serve BED files and API to explore pipeline outputs

The last part of the tutorial consists on running a local instance of `bedhost` (a REST API for `bedstat` and `bedbuncher` produced outputs) in order to explore plots, statistics and download pipeline outputs. To run `bedhost`, we'll pip install the package from the previously cloned repository:

In [ ]:
pip install bedhost/. --user > bedhost_log.txt

To start `bedhost`, we simply need to run the following command passing the location of the bedbase configuration file to the `-c` flag.  

In [ ]:
bedhost serve -c  $BBTUTORIAL/bedbase/tutorial_files/bedbase_configuration.yaml

If we have stored the path to the bedbase config in the environment variable `$BEDBASE` (suggested), it's not neccesary to use said flag. 

In [ ]:
bedhost serve 

The `bedhost` API can be opened in the url [http://0.0.0.0:8000](http://0.0.0.0:8000). We can now explore the plots and statistics generated by the `bedstat` and `bedbuncher` pipelines.